In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [18]:
df_train = pd.read_csv("/content/drive/MyDrive/week 4/train.csv")
df_store = pd.read_csv("/content/drive/MyDrive/week 4/store.csv")
df_test =pd.read_csv("/content/drive/MyDrive/week 4/test.csv")
df_sample = pd.read_csv("/content/drive/MyDrive/week 4/sample_submission.csv")

<ipython-input-18-4790dad0ca93>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv("/content/drive/MyDrive/week 4/train.csv")


In [19]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [10]:
df_train.dtypes

,0
Store,int64
DayOfWeek,int64
Date,object
Sales,int64
Customers,int64
Open,int64
Promo,int64
StateHoliday,object
SchoolHoliday,int64


In [8]:
#handling a missing value
df_train.isnull().sum()

,0
Store,0
DayOfWeek,0
Date,0
Sales,0
Customers,0
Open,0
Promo,0
StateHoliday,0
SchoolHoliday,0


In [21]:
#convert the date column to datetime format
df_train['Date'] = pd.to_datetime(df_train['Date'])


#extracting datetime features
df_train['year']= df_train['Date'].dt.year
df_train['month']= df_train['Date'].dt.month
df_train['day']= df_train['Date'].dt.day
df_train['IsWeekend'] = df_train['DayOfWeek'].apply(lambda x: 1 if x>=5 else 0)
df_train['isMonthStart'] = df_train['Date'].apply(lambda x: 1 if x.is_month_start else 0)
df_train['isMonthEnd'] = df_train['Date'].apply(lambda x: 1 if x.is_month_end else 0)
df_train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,IsWeekend,isMonthStart,isMonthEnd
0,1,5,2015-07-31,5263,555,1,1,0,1,2015,7,31,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1,2015,7,31,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1,2015,7,31,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,2015,7,31,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1,2015,7,31,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1,2013,1,1,0,1,0
1017205,1112,2,2013-01-01,0,0,0,0,a,1,2013,1,1,0,1,0
1017206,1113,2,2013-01-01,0,0,0,0,a,1,2013,1,1,0,1,0
1017207,1114,2,2013-01-01,0,0,0,0,a,1,2013,1,1,0,1,0


In [22]:
#converting catagorical data
df_train = pd.get_dummies(df_train, columns=['StateHoliday'], drop_first=True, dtype= 'int')
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,SchoolHoliday,year,month,day,IsWeekend,isMonthStart,isMonthEnd,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c
0,1,5,2015-07-31,5263,555,1,1,1,2015,7,31,1,0,1,1,0,0,0
1,2,5,2015-07-31,6064,625,1,1,1,2015,7,31,1,0,1,1,0,0,0
2,3,5,2015-07-31,8314,821,1,1,1,2015,7,31,1,0,1,1,0,0,0
3,4,5,2015-07-31,13995,1498,1,1,1,2015,7,31,1,0,1,1,0,0,0
4,5,5,2015-07-31,4822,559,1,1,1,2015,7,31,1,0,1,1,0,0,0


In [25]:
# Create rolling averages and lag features for sales
df_train['Sales_Rolling7'] = df_train['Sales'].rolling(window=7, min_periods=1).mean()
df_train['Sales_Lag1'] = df_train['Sales'].shift(1)
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,SchoolHoliday,year,month,day,IsWeekend,isMonthStart,isMonthEnd,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,Sales_Rolling7,Sales_Lag1
0,1,5,2015-07-31,5263,555,1,1,1,2015,7,31,1,0,1,1,0,0,0,5263.0,NaN
1,2,5,2015-07-31,6064,625,1,1,1,2015,7,31,1,0,1,1,0,0,0,5663.5,5263.0
2,3,5,2015-07-31,8314,821,1,1,1,2015,7,31,1,0,1,1,0,0,0,6547.0,6064.0
3,4,5,2015-07-31,13995,1498,1,1,1,2015,7,31,1,0,1,1,0,0,0,8409.0,8314.0
4,5,5,2015-07-31,4822,559,1,1,1,2015,7,31,1,0,1,1,0,0,0,7691.6,13995.0


In [26]:
# Select numeric columns
numeric_cols = ['Sales', 'Customers', 'Sales_Rolling7', 'Sales_Lag1']

# Scale using StandardScaler
scaler = StandardScaler()
df_train[numeric_cols] = scaler.fit_transform(df_train[numeric_cols])


In [27]:
#print the preprocessed dataset

df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,SchoolHoliday,year,month,day,IsWeekend,isMonthStart,isMonthEnd,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,Sales_Rolling7,Sales_Lag1
0,1,5,2015-07-31,-0.132683,-0.168269,1,1,1,2015,7,31,1,0,1,1,0,0,0,-0.168715,NaN
1,2,5,2015-07-31,0.075373,-0.017540,1,1,1,2015,7,31,1,0,1,1,0,0,0,-0.036441,-0.132684
2,3,5,2015-07-31,0.659800,0.404499,1,1,1,2015,7,31,1,0,1,1,0,0,0,0.255353,0.075372
3,4,5,2015-07-31,2.135414,1.862258,1,1,1,2015,7,31,1,0,1,1,0,0,0,0.870319,0.659799
4,5,5,2015-07-31,-0.247231,-0.159656,1,1,1,2015,7,31,1,0,1,1,0,0,0,0.633382,2.135414


In [28]:
# Save the preprocessed dataset
df_train.to_csv("preprocessed_store_sales.csv", index=False)
